In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms, datasets, models
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Define the path to your dataset
path = '/home/rishabh.mondal/R_P_project/Faulty_solar_panel'

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

# Define the number of splits for cross-validation
n_splits = 4  # 4-fold cross-validation

# Initialize stratified k-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [3]:
model = models.alexnet(pretrained=True)
model

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the Resnet18 model
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

Fold: 1, Epoch 10/50, Train Loss: 1.6481, Train Accuracy: 0.2836, Val Loss: 1.6650, Val Accuracy: 0.2613, Precision: 0.1387, Recall: 0.2613, F1 Score: 0.1622


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 1, Epoch 20/50, Train Loss: 1.5941, Train Accuracy: 0.3680, Val Loss: 1.5577, Val Accuracy: 0.3423, Precision: 0.2318, Recall: 0.3423, F1 Score: 0.2162


...

Fold: 3, Epoch 30/50, Train Loss: 1.5446, Train Accuracy: 0.3313, Val Loss: 1.5880, Val Accuracy: 0.3167, Precision: 0.2922, Recall: 0.3167, F1 Score: 0.2650


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 3, Epoch 40/50, Train Loss: 1.3170, Train Accuracy: 0.4563, Val Loss: 1.4447, Val Accuracy: 0.4027, Precision: 0.4315, Recall: 0.4027, F1 Score: 0.3735
Fold: 3, Epoch 50/50, Train Loss: 0.9755, Train Accuracy: 0.6175, Val Loss: 1.1533, Val Accuracy: 0.5701, Precision: 0.6057, Recall: 0.5701, F1 Score: 0.5590
Training Fold 4/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

Fold: 4, Epoch 10/50, Train Loss: 1.2283, Train Accuracy: 0.5286, Val Loss: 1.3430, Val Accuracy: 0.4570, Precision: 0.4749, Recall: 0.4570, F1 Score: 0.4346
Fold: 4, Epoch 20/50, Train Loss: 0.8706, Train Accuracy: 0.6883, Val Loss: 1.1256, Val Accuracy: 0.6199, Precision: 0.6067, Recall: 0.6199, F1 Score: 0.5957
Fold: 4, Epoch 30/50, Train Loss: 0.4207, Train Accuracy: 0.8464, Val Loss: 1.0529, Val Accuracy: 0.7421, Precision: 0.7479, Recall: 0.7421, F1 Score: 0.7350
Fold: 4, Epoch 40/50, Train Loss: 0.3499, Train Accuracy: 0.8780, Val Loss: 1.2286, Val Accuracy: 0.7285, Precision: 0.7437, Recall: 0.7285, F1 Score: 0.7221
Fold: 4, Epoch 50/50, Train Loss: 0.1794, Train Accuracy: 0.9413, Val Loss: 1.3709, Val Accuracy: 0.7014, Precision: 0.7104, Recall: 0.7014, F1 Score: 0.6997


In [5]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the ResNet18 model
    model = models.alexnet(pretrained=False)
    model.classifier[6] = nn.Linear(4096, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 1, Epoch 10/50, Train Loss: 1.7355, Train Accuracy: 0.2338, Val Loss: 1.7255, Val Accuracy: 0.2342, Precision: 0.0549, Recall: 0.2342, F1 Score: 0.0889


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 1, Epoch 20/50, Train Loss: 1.6491, Train Accuracy: 0.2805, Val Loss: 1.7149, Val Accuracy: 0.2838, Precision: 0.1305, Recall: 0.2838, F1 Score: 0.1747
Fold: 1, Epoch 30/50, Train Loss: 1.3525, Train Accuracy: 0.4842, Val Loss: 1.4652, Val Accuracy: 0.4009, Precision: 0.4022, Recall: 0.4009, F1 Score: 0.3789
Fold: 1, Epoch 40/50, Train Loss: 1.0046, Train Accuracy: 0.6063, Val Loss: 1.3165, Val Accuracy: 0.5360, Precision: 0.5578, Recall: 0.5360, F1 Score: 0.5323
Fold: 1, Epoch 50/50, Train Loss: 0.6842, Train Accuracy: 0.7391, Val Loss: 1.3573, Val Accuracy: 0.5541, Precision: 0.5821, Recall: 0.5541, F1 Score: 0.5527
Training Fold 2/4
3 1


...

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 4, Epoch 30/50, Train Loss: 1.7516, Train Accuracy: 0.2425, Val Loss: 1.7290, Val Accuracy: 0.2308, Precision: 0.0533, Recall: 0.2308, F1 Score: 0.0865


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 4, Epoch 40/50, Train Loss: 1.7849, Train Accuracy: 0.2455, Val Loss: 1.6814, Val Accuracy: 0.2624, Precision: 0.3772, Recall: 0.2624, F1 Score: 0.1697
Fold: 4, Epoch 50/50, Train Loss: 1.7058, Train Accuracy: 0.2530, Val Loss: 1.7876, Val Accuracy: 0.1448, Precision: 0.0328, Recall: 0.1448, F1 Score: 0.0442


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
